# VL 11: CNNs

This Jupyter Notebook demonstrates the process of loading, preprocessing, and modeling mortality rate data using a deep neural network.

## Data Loading and Preprocessing

1. **Importing Libraries**:
    - We import necessary libraries including `numpy`, `pandas`, and various modules from `tensorflow.keras` for building the neural network.
    - We also import `LabelEncoder` from `sklearn.preprocessing` for encoding categorical data.

2. **Loading Data**:
    - We load the mortality rates data from a CSV file into a pandas DataFrame named `TrainData`.

3. **Extracting Features and Target**:
    - We extract the `Year`, `Age`, `Country`, and `sex` columns as features and the `log_mx` column as the target variable.
    - We reshape the extracted features to be used in the neural network.

4. **Encoding Categorical Data**:
    - We use `LabelEncoder` to encode the `Country` and `sex` columns into numerical values.

5. **Setting Limits**:
    - We determine the maximum values for `Age`, `Country`, and `Gender` to set the input dimensions for the embedding layers.

## Model Building

1. **Defining Inputs**:
    - We define the input layers for the rates, country, and gender.

2. **Embedding Layers**:
    - We create embedding layers for the `Country` and `Gender` inputs to convert them into dense vectors.

3. **Convolutional Layers**:
    - We apply a series of convolutional, pooling, batch normalization, and dropout layers to the rates input to extract features.

4. **Concatenation and Dense Layers**:
    - We concatenate the outputs of the convolutional layers and the embedding layers.
    - We apply dropout and dense layers to the concatenated output to generate the final forecast rates.

5. **Compiling the Model**:
    - We compile the model using the Adam optimizer and mean squared error loss function.

## Model Training

1. **Early Stopping**:
    - We add an early stopping callback to prevent overfitting by monitoring the validation loss.

2. **Generating Dummy Data**:
    - We generate dummy data for the rates input to simulate the training process.
    - **This is the part I don't understand a bit: What good does that do, why do I need it?**

3. **Fitting the Model**:
    - We fit the model on the training data with a validation split and the early stopping callback.

This workflow provides a comprehensive approach to building and training a deep neural network for forecasting mortality rates based on various features.

In [6]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Embedding, Flatten, Conv1D, MaxPooling1D, BatchNormalization, Dropout, Dense, Reshape, Concatenate
from tensorflow.keras.models import Model

from sklearn.preprocessing import LabelEncoder

In [7]:
TrainData = pd.read_csv('../VL10-DeepNeuralNetworks/mortality_rates.csv')
YearData = TrainData['Year'].values.reshape(-1, 1)
AgeData = TrainData['Age'].values.reshape(-1, 1)
_CountryData = TrainData['Country'].values
_GenderData = TrainData["sex"].values
# Encode country strings to integers
label_encoder = LabelEncoder()
CountryData = label_encoder.fit_transform(_CountryData).reshape(-1, 1)
GenderData = label_encoder.fit_transform(_GenderData).reshape(-1, 1)

ageLim = AgeData.max()+1
CountryLim = CountryData.max()+1
GenderLim = GenderData.max()+1

print("Age limit: ", ageLim, "Country limit", CountryLim, "Gender limit", GenderLim) #I had to modify the code as I seemed to have a country more than the Wuethrich dataset. Gender and age however are the same.

TargetData = TrainData['log_mx'].values.reshape(-1, 1)

Age limit:  101 Country limit 42 Gender limit 2


In [27]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the inputs
rates = Input(shape=(10, 100), dtype='float32', name='rates')
Country = Input(shape=(1,), dtype='int32', name='Country')
Gender = Input(shape=(1,), dtype='int32', name='Gender')

# Embedding layers
Country_embed = Embedding(input_dim=CountryLim, output_dim=5)(Country)
Country_embed = Flatten(name='Country_embed')(Country_embed)

Gender_embed = Embedding(input_dim=GenderLim, output_dim=5)(Gender)
Gender_embed = Flatten(name='Gender_embed')(Gender_embed)

# Convolutional layers
conv = Conv1D(filters=32, kernel_size=3, activation='linear', padding='causal')(rates)
conv = MaxPooling1D(pool_size=2)(conv)
conv = BatchNormalization()(conv)
conv = Dropout(rate=0.35)(conv)
conv = Flatten()(conv)

# Concatenate and dense layers
decoded = Concatenate()([conv, Country_embed, Gender_embed])
decoded = Dropout(rate=0.4)(decoded)
decoded = Dense(units=100, activation='sigmoid')(decoded)
decoded = Reshape((1, 100), name='forecast_rates')(decoded)

# Compile the model
model = Model(inputs=[rates, Country, Gender], outputs=decoded)
model.compile(optimizer='adam', loss='mse')

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Generate dummy data for rates
rates_data = np.random.rand(len(CountryData), 10, 100).astype('float32')

# Fit the model
model.fit(
    [rates_data, CountryData, GenderData], 
    TargetData, 
    epochs=10, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[early_stopping]
)

Epoch 1/10
9782/9782 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - loss: 29.4749 - val_loss: 28.6259
Epoch 2/10
9782/9782 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - loss: 29.3851 - val_loss: 28.6259
Epoch 3/10
9782/9782 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 29.4172 - val_loss: 28.6259
Epoch 4/10
9782/9782 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 29.3692 - val_loss: 28.6259
